In [1]:
import math
import time
import numpy as np
from robocup import field
from robocup import vision
from robocup import switches as s
from robocup import movement as mov
from robocup import commands as comm
from robocup import localization as loc
#from filterpy.kalman import KalmanFilter
from multiprocessing import Process, Value
#from filterpy.common import Q_discrete_white_noise
from robocup import bluetooth_communication as blu


robot_angle = 225

In [2]:
xnum = Value('d', 0.0)
ynum = Value('d', 0.0)
anglenum = Value('d', 0.0)
distnum = Value('d', 0.0)
sizenum = Value('d', 0.0)
recnum = Value('d', 0.0)
erasenum = Value('d', 0.0)
#vision.detect_the_ball(anglenum, distnum, sizenum, recnum, True)
p1 = Process(target=loc.localize, args=(robot_angle, xnum, ynum, False))
p2 = Process(target=vision.detect_the_ball, args=(anglenum, distnum, sizenum, recnum, erasenum, False))
p1.start()
p2.start()

640x480
640x480


Too many measurments in the input buffer: 574/500. Clearing buffer...
Too many measurments in the input buffer: 501/500. Clearing buffer...
Too many measurments in the input buffer: 615/500. Clearing buffer...


In [3]:
erasenum.value = 1

In [4]:
erasenum.value = 0

In [5]:
recnum.value = 1

In [6]:
recnum.value = 0

In [ ]:
while s.state(0):
    a = 0
comm.set_orientation()
while not s.state(0):
    a = 0
comm.start()
x = field.x / 2 + 0.01
y = field.y / 2 + 0.012
seeball = 0
robot_speed = 0.1
goalie = False
goalie_last_time = 0

while True:
    xc = xnum.value
    yc = ynum.value
    angle = anglenum.value
    ball_size = sizenum.value
    dist = distnum.value   

    seeball += 1 if ball_size > 0 else -1 
    if seeball > 3:
        seeball = 3
    if seeball < -1:
        seeball = -1
        
    if (xc != -1):
        x = xc
    if (yc != -1): 
        y = yc
        
    if s.state(2):
        goalie_last_time = goalief(angle, ball_size, dist, goalie_last_time)
    else:
        attacker()
    #print(x, y, robot_speed, angle, ba, seeball)#,  end = '\r')
    time.sleep(0.005)
    s.check_pause()
    if not s.state(3):
        break

In [7]:
def attacker():
    if seeball > 2:
        robot_speed = 0.3
        ba = mov.robot_angle_to_catch_ball(angle, x, y)
        _, ba = mov.flatten_angle_attacker(upper_dist=800, x=x, y=y, ant=ba, speed=robot_speed)
        robot_speed, angle_out = mov.adaptive_speed_line(max_speed=0.4 , min_speed=0, upper_dist=800, x=x, y=y, ant=(ba)%360)
        
        min_speed = robot_speed
        if angle_out is None:
            comm.set_moving_angle(ba, min_speed)
        else:
            comm.set_moving_angle(angle_out, robot_speed)
    else:
        mov.move_to(x, y, field.x/2, field.y/2, 0.6)    

In [8]:
def goalief(ball_angle, ball_size, ball_distance, goalie_last_time):
    robot_speed = 0.3
    #print (time.time() - goalie_last_time, ball_distance, seeball)
    if seeball > 2:
        dirr = math.sin(math.radians(ball_angle))
        ba = 0 if ball_angle<0 else 180
        robot_max_speed = 0.4*abs(dirr)
        #ba = mov.robot_angle_to_catch_ball(ball_angle, x, y)
        #print("ba1=", ba)
        ##_, ba = mov.flatten_angle_goalie(upper_dist=800, x=x, y=y, ant=ba, speed=robot_speed)
        #print("ba flat=", ba)
        #robot_speed, ba = mov.goalie_vector_transform(x, y, ba, robot_max_speed)
        ##robot_speed, ba = mov.goalie_vector_transform(x, y, ba, robot_speed)
        robot_speed, angle_out = mov.adaptive_speed_line_goalie(max_speed=robot_max_speed , min_speed=0, upper_dist=200, x=x, y=y, ant=(ba)%360)
        
        
        if ball_distance > 350:
            goalie_last_time = time.time()  
        if time.time() - goalie_last_time  > 7:
            #robot_speed, ba = mov.goalie_vector_transform(x, y, ba, robot_speed)
            attacker()
            return goalie_last_time
        min_speed = robot_speed
        if angle_out is None:
            comm.set_moving_angle(ba, min_speed)
        else:
            comm.set_moving_angle(angle_out, robot_speed)
    else:
        mov.move_to(x, y, field.x/2, 520, 0.7)
        goalie_last_time = time.time()
    return goalie_last_time    
        
        

In [5]:
mov.goalie_vector_transform(900, 900, 0, 0)

(0.1902629759044045, 273.01278750418334)

In [15]:
dt = 0.005
kf = KalmanFilter (dim_x=4, dim_z=2)
f.x = np.array([[field.x/2],
                [field.y/2],
                [0.], 
                [0.]])   
    
f.F = np.array([[1., 0., dt, 0.],
                [0., 1., 0., dt],
                [0., 0., 1., 0.],
                [0., 0., 0., 1.]])
    
f.H = np.array([[1., 0., 0., 0.],
                [0., 1., 0., 0.]])
    
f.P *= 1000.
R = np.array([[50., 50.]])
R_nox = np.array([[50000., 50.]])
R_noy = np.array([[50000., 50.]])
R_noxy = np.array([[50000., 50000.]])
f.R = R
f.Q = Q_discrete_white_noise(dim=4, dt=dt, var=0.13)
last_filter_call = time.time()
d_filter_call = 0.005
xf = field.x/2
yf = field.y/2
def filter_loc(was_measurment_updated, xm, ym):
    if was_measurment_updated:
        f.predict()
        f.update(z)
    elif time.time() - last_filter_call > d_filter_call:
        f.predict()
        
    print(f.x) 
    return f.x[0], f.x[1]

NameError: name 'KalmanFilter' is not defined

In [4]:
20*(0>0)

0

In [1]:
from robocup import movement as mov
lines = [
    [260, 260, 570, 260],
    [570, 260, 570, 410],
    [570, 410, 660, 520],
    [660, 520, 1160, 520],
    [1160, 520, 1250, 410],
    [1250, 410, 1250, 260],
    [1250, 260, 1560, 260],
    [1560, 260, 1560, 2170],
    [1560, 2170, 1250, 2170],
    [1250, 2170, 1250, 2020],
    [1250, 2020, 1160, 1910],
    [1160, 1910, 660, 1910],
    [660, 1910, 570, 2020],
    [570, 2020, 570, 2170],
    [570, 2170, 260, 2170],
    [260, 2170, 260, 260]
]

In [3]:
help(mov)

Help on module robocup.movement in robocup:

NAME
    robocup.movement

FUNCTIONS
    adaptive_speed_ball(angle_to_go, ball_dist, speed, ball_close_speed)
    
    adaptive_speed_line(max_speed, min_speed, upper_dist, x, y, ant)
    
    adaptive_speed_rect(max_speed, min_speed, ball_close_speed, upper_dist, lower_dist, x, y, ant, ball_dist)
    
    flatten_angle(upper_dist, x, y, ant)
    
    line_dists(x1, y1)
    
    move_to(x0, y0, x, y)
    
    robot_angle_to_catch_ball(ball_angle, x, y)

DATA
    lines = [[260, 260, 570, 260], [570, 260, 570, 410], [570, 410, 660, 5...

FILE
    /home/pi/Desktop/24.11.2019. - 21h/jupyter/robocup/movement.py




In [14]:
mov.flatten_angle(300, 540, 770, 300, 1)

(-8.752538256722763, 0.6259580211832472)